# Forward auto-diff through Dual Numbers

In [22]:
import numpy as np

Let us define a class `DualNumber` that represents a dual number

$$
a + b \epsilon 
$$

where $a$ is the "real" part and $b$ is the "dual" part.

In [23]:
class DualNumber:
  def __init__(self, real, dual):
    # dual number: 'real' + 'dual' * eps
    self.real = real
    self.dual = dual

  def __repr__(self):
    return repr(self.real) + ' + ' + repr(self.dual) + ' epsilon'

Implement the operator sum `__add__` for this class.

In [24]:
class DualNumber:
  def __init__(self, real, dual):
    # dual number: 'real' + 'dual' * eps
    self.real = real
    self.dual = dual

  def __repr__(self): # overload the print method
    return repr(self.real) + ' + ' + repr(self.dual) + ' epsilon'

  def __add__(self, other): # overload the '+' operator
    return DualNumber(self.real + other.real, self.dual + other.dual)

Define the dual numbers

$$
\begin{split}
x &= 1 + 2 \epsilon \\
y &= 1.5 + 3.1 \epsilon \\
\end{split}
$$

Then, compute $z = x + y$ and display the result.

In [25]:
x = DualNumber(1, 2)
y = DualNumber(1.5, 3.1)

z = x + y

print(z)

2.5 + 5.1 epsilon


Define now the dual number

$$
\begin{split}
x &= 1 + 2 \epsilon \\
\end{split}
$$

and try to compute $w = x + 1$. What is going on?

In [26]:
w = x + 1
print(w)

AttributeError: 'int' object has no attribute 'dual'

Try now to compute $w = 1 + x$ (in this specific order). What is going on this time?

In [27]:
w = 1 + x
print(w)

TypeError: unsupported operand type(s) for +: 'int' and 'DualNumber'

To overcome the above inconvenient, introduce a check (inside the definition of `__add__`) on the type of `other`. Moroever, define the operator `__radd__`, besides `__add__`.

In [28]:
class DualNumber:
  def __init__(self, real, dual):
    # dual number: 'real' + 'dual' * eps
    self.real = real
    self.dual = dual

  def __repr__(self):
    return repr(self.real) + ' + ' + repr(self.dual) + ' epsilon'

  def __add__(self, other):
    # implement the operation "self + other"
    if isinstance(other, DualNumber):
      return DualNumber(self.real + other.real, self.dual + other.dual)
    else:
      return DualNumber(self.real + other, self.dual)

  def __radd__(self, other):
    # implement the operation "other + self"
    return self.__add__(other)

Try again to compute $w = x + 1$

In [29]:
x = DualNumber(1, 2)

w = x + 1
print(w)

2 + 2 epsilon


Try again to compute $w = 1 + x$

In [30]:
w = 1 + x
print(w)

2 + 2 epsilon


Now that we have learnt how to treat the operator "+", let us define the full class `DualNumber`, implementing also the operators "-", "*", "/", "**".

In [31]:
class DualNumber:
  def __init__(self, real, dual):
    # dual number: 'real' + 'dual' * eps
    self.real = real
    self.dual = dual

  def __add__(self, other):
    if isinstance(other, DualNumber):
      return DualNumber(self.real + other.real, self.dual + other.dual)
    else:
      return DualNumber(self.real + other, self.dual)
  
  def __radd__(self, other):
    # implement the operation "other + self"
    return self.__add__(other)

  def __sub__(self, other):
    if isinstance(other, DualNumber):
      return DualNumber(self.real - other.real, self.dual + other.dual)
    else:
      return DualNumber(self.real - other, self.dual)

  def __rsub__(self, other):
    # implement the operation "other - self"
    return DualNumber(other, 0.0) - self

  def __mul__(self, other):
    # implement the operation "self * other"
    if isinstance(other, DualNumber):
      return DualNumber(self.real * other.real, self.real * other.dual + self.dual * other.real)
    else:
      return DualNumber(self.real * other, self.dual * other)

  def __rmul__(self, other):
    # implement the operation "other * self"
    return self.__mul__(other)

  def __truediv__(self, other):
    # implement the operation "self / other"
    if isinstance(other, DualNumber):
      return DualNumber(self.real / other.real, (self.dual * other.real - self.real * other.dual) / (other.real ** 2))
    else:
      return (1 / other) * self

  def __rtruediv__(self, other):
    # implement the operation "other / self"
    return DualNumber(other, 0.0).__truediv__(self)

  def __pow__(self, other):
    if other == 0:
      return 1
    return self * self.__pow__(other - 1)

  def __repr__(self):
    return repr(self.real) + ' + ' + repr(self.dual) + ' epsilon'


Define the dual numbers

$$
\begin{split}
x &= 1 + 2 \epsilon \\
y &= 1.5 + 3.1 \epsilon \\
\end{split}
$$

Then, compute the result of the following operations:
- $x + y$
- $x - y$
- $x y$
- $x / y$
- $x + 1$
- $2 x$
- $x ^ 3$

Define now the functions `my_sin`, `my_cos` and `my_exp`, implementing the operations sinus, cosinus and exponential, respectively.

In [32]:
def my_sin(x):
  if isinstance(x, DualNumber):
    return DualNumber(np.sin(x.real), x.dual * np.cos(x.real))
  else:
    return np.sin(x)

def my_cos(x):
  if isinstance(x, DualNumber):
    return DualNumber(np.cos(x.real), -x.dual * np.sin(x.real))
  else:
    return np.cos(x)

def my_exp(x): # TO DO
  if isinstance(x, DualNumber):
    return DualNumber(np.exp(x.real), np.exp(x.real) * x.dual)
  else:
    return np.exp(x)

Define the dual number

$$
\begin{split}
x &= 1 + 2.3 \epsilon \\
\end{split}
$$

Then, compute the result of the following operations:
- $\sin(x)$
- $\exp(x)$

Define now a function `auto_diff` that, given a function $f \colon \mathbb{R} \to \mathbb{R}$ and a real number $x$, returns $f'(x)$, exploiting the class `DualNumber`. The function must have the following signature:
```python
def auto_diff(f, x):
  ...
```


In [33]:
def auto_diff(f, x):
  return f(DualNumber(x, 1)).dual

Consider the function 

$$
f(x) = x \sin(x^2)
$$

and use the function implemented above to compute $f'(x_0)$ for $x_0 = 0.13$. Compare the result with the analytical solution and compute the relative error.



In [34]:
func = lambda x : x * my_sin(x ** 2)
x0 = auto_diff(func, 0.13)
x0 # il valore esatto è: 0.050694 --> ottimo risultato

0.050694368849202455

Repeat the previous point, this time by computing the numerical derivative (i.e. through finite differences).

In [35]:
import scipy.misc
df_FD = scipy.misc.derivative(func, 0.13, dx=1e-6) # funzione deprecata --> non importa
print('f\'(x0) (FD): %f' % df_FD)
#print('err (FD): %e' % (abs(df_FD - df_ex)/abs(df_ex))) --> stima dell'errore, df_ex è il valore reale della derivata
# il valore reale lo puoi calcolare manualmente, facendo la derivata della funzione in questione

f'(x0) (FD): 0.050694


/var/folders/_v/q6j8tl0d2cb93y5h0_d24pgr0000gn/T/ipykernel_2281/2720248378.py:2: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  df_FD = scipy.misc.derivative(func, 0.13, dx=1e-6) # funzione deprecata --> non importa


Repeat the previous point, this time by computing the symbolic derivative (module `sympy` = **sym**bolic **py**thon)

In [36]:
import sympy
x = sympy.symbols('x')
func_sym = x * sympy.sin(x ** 2)

dfunc_sym = sympy.diff(func_sym, x)
print(dfunc_sym)
df_sy = dfunc_sym.subs(x, 0.13)

print('f\'(x0) (sy): %f' % df_sy)
#print('err (sy): %e' % (abs(df_sy - df_ex)/abs(df_ex))) --> stima dell'errore, df_ex è il valore reale della derivata

# non riesco a installare questo package --> comunque il risultato che otteniamo lo stesso risultato

2*x**2*cos(x**2) + sin(x**2)
f'(x0) (sy): 0.050694


Evaluate and compare the execution time of the different approaches.
To compute the execution time of a line of code, prepend IPython [magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html) `%timeit` to the line.

Example:
```python
%timeit np.random.rand(1000)
```

In [37]:
%timeit x0 = auto_diff(func, 0.13) # praticamente è come valutare la funzione derivata reale nel punto --> ottimale

4.6 µs ± 47.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [38]:
%timeit x0 = scipy.misc.derivative(func, 0.13, dx=1e-6) # come vedi, questo approccio è molto più lento

<magic-timeit>:1: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools


16 µs ± 155 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Consider now the function 
$$
f(x) = \frac{1}{x^5}
$$
compute the derivative in the point $x_0 = 10^{-2}$ with AD and FD and compare the results with the exact solution.

In [39]:
func = lambda x : 1 / (x ** 5)
point = 10 ** -2

print(auto_diff(func, point))
print(scipy.misc.derivative(func, point, dx=1e-6))
# exact solution: -5000000000000.000000

-4999999999999.999
-5000000349996.567


/var/folders/_v/q6j8tl0d2cb93y5h0_d24pgr0000gn/T/ipykernel_2281/4238742683.py:5: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  print(scipy.misc.derivative(func, point, dx=1e-6))


In [ ]:
# il metodo dei Dual Number è il migliore --> molto usato nel deep learning